In [2]:
import joblib
from tensorflow import keras
from tensorflow.keras import layers, models

In [29]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import cv2

results = pd.read_csv("/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Training_GroundTruth.csv")
images = results["image"].to_list()

image_vectors = []
# Проход по каждому изображению
for image_file in tqdm(images):
    # Загрузка изображения
    image_path = f"/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Training_Input/{image_file}.jpg"
    image = cv2.imread(image_path)
    image = cv2.resize(image, None, fx=0.3, fy=0.3)
    image_vectors.append(image)
    
X_train = np.array(image_vectors)
y_train = np.asarray(results.drop("image", axis=1))

100%|██████████| 10015/10015 [00:34<00:00, 290.38it/s]


In [30]:
results = pd.read_csv("/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Test_GroundTruth.csv")
images = results["image"].to_list()

image_vectors = []
# Проход по каждому изображению
for image_file in tqdm(images):
    # Загрузка изображения
    image_path = f"/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Test_Input/{image_file}.jpg"
    image = cv2.imread(image_path)
    image = cv2.resize(image, None, fx=0.3, fy=0.3)
    image_vectors.append(image)
    
X_test = np.array(image_vectors)
y_test = np.asarray(results.drop("image", axis=1))

100%|██████████| 1512/1512 [00:05<00:00, 296.37it/s]


In [31]:
results = pd.read_csv("/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Validation_GroundTruth.csv")
images = results["image"].to_list()

image_vectors = []
# Проход по каждому изображению
for image_file in tqdm(images):
    # Загрузка изображения
    image_path = f"/Users/aleksey/PycharmProjects/MediScan/Aleksey/ISIC2018_Task3_Validation_Input/{image_file}.jpg"
    image = cv2.imread(image_path)
    image = cv2.resize(image, None, fx=0.3, fy=0.3)
    image_vectors.append(image)
    
X_val = np.array(image_vectors)
y_val = np.asarray(results.drop("image", axis=1))

100%|██████████| 193/193 [00:00<00:00, 290.31it/s]


In [43]:
# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (2, 2), activation='relu', input_shape=(image.shape[0], image.shape[1], image.shape[2])))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (2, 2), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy", 
                       keras.metrics.Precision(),
                       keras.metrics.Recall(),
                       keras.metrics.AUC(),
                       ])

# Train the model
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))

Epoch 1/5
313/313 [==============================] - 57s 179ms/step - loss: 17.7050 - accuracy: 0.6529 - precision_8: 0.7978 - recall_8: 0.4845 - auc_2: 0.8938 - val_loss: 0.9292 - val_accuracy: 0.6373 - val_precision_8: 0.8387 - val_recall_8: 0.5389 - val_auc_2: 0.9267
Epoch 2/5
313/313 [==============================] - 87s 278ms/step - loss: 0.9255 - accuracy: 0.6857 - precision_8: 0.8367 - recall_8: 0.5457 - auc_2: 0.9302 - val_loss: 0.9227 - val_accuracy: 0.6528 - val_precision_8: 0.8707 - val_recall_8: 0.5233 - val_auc_2: 0.9275
Epoch 3/5
313/313 [==============================] - 98s 314ms/step - loss: 0.8029 - accuracy: 0.7142 - precision_8: 0.8474 - recall_8: 0.6011 - auc_2: 0.9456 - val_loss: 0.9582 - val_accuracy: 0.6528 - val_precision_8: 0.8134 - val_recall_8: 0.5648 - val_auc_2: 0.9215
Epoch 4/5
313/313 [==============================] - 94s 302ms/step - loss: 0.7453 - accuracy: 0.7357 - precision_8: 0.8544 - recall_8: 0.6286 - auc_2: 0.9536 - val_loss: 0.8837 - val_accur

In [44]:
loss, acc, precision, recall, auc = model.evaluate(X_test, y_test)

48/48 [==============================] - 3s 65ms/step - loss: 1.4633 - accuracy: 0.6184 - precision_8: 0.7038 - recall_8: 0.5562 - auc_2: 0.8835


In [67]:
predictions = model.predict(X_test)

48/48 [==============================] - 2s 46ms/step


In [68]:
predictions = np.round(predictions, decimals=5)

In [73]:
class_dict = {"MEL": 0., "NV": 0., "BCC": 0., "AKIEC": 0., "BKL": 0., "DF": 0., "VASC": 0.}
c = 0
for key, value in class_dict.items():
    class_dict[key] = predictions[2][c]
    c += 1

In [74]:
class_dict

{'MEL': 0.06582,
 'NV': 0.2747,
 'BCC': 0.2311,
 'AKIEC': 0.03609,
 'BKL': 0.316,
 'DF': 0.07463,
 'VASC': 0.00166}

In [47]:
y_test

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [64]:
joblib.dump(model, '/Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP model/cnn_model.pkl')

['/Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP model/cnn_model.pkl']

In [4]:
model = joblib.load('/Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP models/CNN Model/cnn_model.pkl')

In [6]:
keras.saving(model, "/Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP models/CNN Model/cnn_model.pkl")

TypeError: 'module' object is not callable

In [7]:
model.save('/Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP models/CNN Model/cnn_model_v2')

INFO:tensorflow:Assets written to: /Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP models/CNN Model/cnn_model_v2/assets


INFO:tensorflow:Assets written to: /Users/aleksey/PycharmProjects/MediScan/Aleksey/MVP models/CNN Model/cnn_model_v2/assets
